In [1]:
# Core
import numpy as np
import pandas as pd

# Astronomy
import rebound

In [2]:
# Local imports
import kepler_sieve
from db_utils import sp2df

In [3]:
from asteroid_element import get_ast_elts, make_sim_asteroids, make_test_elements
from orbital_element import anomaly_E2f, anomaly_f2E, anomaly_E2M, danby_guess, danby_iteration, elt2pos
from orbital_element import get_test_elements, angle_distance, report_test, test_E2f, test_E2M, test_M2E, test_all
from asteroid_data import ast_add_sun_vectors

In [4]:
from planets_interp import get_earth_pos, get_earth_vectors, get_earth_elt, get_sun_vectors

In [7]:
elts = get_test_elements()
ast_add_sun_vectors(elts)

In [8]:
elts

,TimeID,AsteroidID,mjd,qx,qy,qz,vx,vy,vz,a,...,Omega,omega,f,M,sun_qx,sun_qy,sun_qz,sun_vx,sun_vy,sun_vz
0,84960000,1,59000.0,2.200915,-1.931843,-0.467561,0.006340,0.007130,-0.000945,2.767657,...,1.401272,1.286860,-3.401546,2.839411,-0.00504,0.007027,0.000058,-0.000008,-0.000004,2.328685e-07
1,84960000,2,59000.0,0.662691,-2.706223,1.817727,0.008356,0.000283,-0.000904,2.773841,...,3.019851,-0.869132,2.742191,2.530303,-0.00504,0.007027,0.000058,-0.000008,-0.000004,2.328685e-07
2,84960000,3,59000.0,-2.901474,-1.192233,0.390143,0.001943,-0.008331,0.001812,2.668285,...,2.964468,-1.953614,2.535135,2.189260,-0.00504,0.007027,0.000058,-0.000008,-0.000004,2.328685e-07
3,84960000,4,59000.0,-0.240387,2.551044,-0.047390,-0.010162,-0.001270,0.001274,2.362014,...,1.811810,2.633265,-2.783176,-2.716995,-0.00504,0.007027,0.000058,-0.000008,-0.000004,2.328685e-07
4,84960000,5,59000.0,-2.063769,0.498221,0.084128,-0.003827,-0.012194,0.001120,2.574037,...,2.470881,-0.023590,0.461775,0.311477,-0.00504,0.007027,0.000058,-0.000008,-0.000004,2.328685e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9988,84960000,9995,59000.0,1.929299,1.989779,0.047063,-0.006940,0.006451,0.000343,2.390249,...,0.363924,-2.826248,3.260392,-2.979609,-0.00504,0.007027,0.000058,-0.000008,-0.000004,2.328685e-07
9989,84960000,9996,59000.0,-0.850272,-2.870477,0.281759,0.009476,-0.000501,0.000683,2.796605,...,-2.629937,2.898821,-2.120904,-1.682728,-0.00504,0.007027,0.000058,-0.000008,-0.000004,2.328685e-07
9990,84960000,9997,59000.0,-2.235940,-1.662699,-0.104958,0.005379,-0.008184,-0.000224,2.545674,...,-0.427796,1.579063,-3.649961,2.514484,-0.00504,0.007027,0.000058,-0.000008,-0.000004,2.328685e-07
9991,84960000,9998,59000.0,1.949042,-0.287275,0.015234,0.001389,0.012640,0.000860,2.160961,...,-0.262023,0.578132,-0.465334,-0.386152,-0.00504,0.007027,0.000058,-0.000008,-0.000004,2.328685e-07


In [7]:
# test_all()

In [9]:
# Unpack elements
a = elts.a.values
e = elts.e.values
inc = elts.inc.values
Omega = elts.Omega.values
omega = elts.omega.values
f = elts.f.values
M = elts.M.values

In [10]:
# Position of the sun
cols_q_sun = ['sun_qx', 'sun_qy', 'sun_qz']
q_sun = elts[cols_q_sun].values

In [11]:
q2 = q_sun + elt2pos(a=a, e=e, inc=inc, Omega=Omega, omega=omega, f=f)

In [12]:
cols_q = ['qx', 'qy', 'qz']
q = elts[cols_q].values

In [13]:
dq = q2 - q
err = np.sqrt(np.sum(np.square(dq), axis=1))
err

array([1.62506466e-15, 1.90039570e-15, 1.19574679e-15, ...,
       3.77190001e-15, 3.22771397e-15, 3.47405305e-15])

In [14]:
dq

array([[ 0.00000000e+00, -2.22044605e-16,  1.60982339e-15],
       [-1.88737914e-15,  0.00000000e+00,  2.22044605e-16],
       [-4.44089210e-16,  8.88178420e-16, -6.66133815e-16],
       ...,
       [-4.44089210e-16,  4.44089210e-16,  3.71924713e-15],
       [ 2.22044605e-16,  3.21964677e-15,  5.20417043e-17],
       [-3.44169138e-15,  4.44089210e-16, -1.63064007e-16]])

In [14]:
# calculate E from f and e
E = anomaly_f2E(f=f, e=e)

In [15]:
# # Calculate the distance from the center, r; SSD equation 2.20
# r = a * (1.0 - np.square(e)) / (1 + e * np.cos(f))

# # Calculate intermediate results used for angular rotations
# theta = omega + f # the angle in the elliptic plane, measured from the reference direction
# cos_inc = np.cos(inc)
# sin_inc = np.sin(inc)
# cos_Omega = np.cos(Omega)
# sin_Omega = np.sin(Omega)
# cos_theta = np.cos(theta)
# sin_theta = np.sin(theta)

# # The cartesian position coordinates; see SSD equation 2.122
# qx = r * (cos_Omega * cos_theta - sin_Omega * sin_theta * cos_inc)
# qy = r * (sin_Omega * cos_theta - cos_Omega * sin_theta * cos_inc)
# qz = r * (sin_theta * sin_inc)

In [16]:
qx

NameError: name 'qx' is not defined

In [22]:
q = np.stack([qx, qy, qz], axis=1)

In [23]:
q.shape

(9993, 3)